In [ ]:
# Cell 1: Environment Setup
!pip install --upgrade transformers datasets peft accelerate bitsandbytes scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: peft
 

In [ ]:
# Cell 2: Import Libraries and Mount Google Drive
import os
import glob
import pandas as pd
import numpy as np
import torch
from google.colab import drive

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 3: Data Loading and Cleaning (Corrected)

import os
import glob
import pandas as pd

# --- Configuration ---
DATA_DIR = "/content/drive/MyDrive/CSCI594Project/"

# --- Loading ---
csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
if not csv_files:
    raise ValueError(f"No CSV files found in the specified directory: {DATA_DIR}")

df_list = [pd.read_csv(file) for file in csv_files]
full_df = pd.concat(df_list, ignore_index=True)

print(f"Loaded {len(full_df)} rows from {len(csv_files)} files.")
print("Original DataFrame columns:", full_df.columns.tolist())
print("\nOriginal label distribution:\n", full_df['label'].value_counts())

# --- Cleaning & Normalization ---
# Define label mappings
label2id = {'NORMAL': 0, 'OFFENSIVE': 1, 'HATE_SPEECH': 2}
id2label = {0: 'NORMAL', 1: 'OFFENSIVE', 2: 'HATE_SPEECH'}

# Keep only the necessary columns
cleaned_df = full_df[['text', 'label']].copy()

# 1. *** THE LABEL FIX: NORMALIZE LABELS ***
# Convert all labels to string and then to uppercase
cleaned_df['label'] = cleaned_df['label'].astype(str).str.upper()

# 2. Map string labels to integers
# Invalid labels (like 'API_FAILURE') will become NaN
cleaned_df['label'] = cleaned_df['label'].map(label2id)

# 3. *** THE TEXT FIX: Drop rows missing text OR a valid label ***
# This drops rows where 'text' is NaN OR 'label' is NaN
cleaned_df.dropna(subset=['text', 'label'], inplace=True)

# 4. Convert label to integer
cleaned_df['label'] = cleaned_df['label'].astype(int)

print(f"\nFinal DataFrame shape: {cleaned_df.shape}")
print("Final label distribution:\n", cleaned_df['label'].value_counts())
print("\nSample of cleaned data:")
print(cleaned_df.head())

Loaded 4220 rows from 7 files.
Original DataFrame columns: ['author_hash', 'text_redacted', 'label', 'comment', 'text', 'language_category', 'user']

Original label distribution:
 label
OFFENSIVE      1383
NORMAL         1063
normal          841
HATE_SPEECH     787
offensive       106
hate_speech      38
API_FAILURE       1
label             1
Name: count, dtype: int64

Final DataFrame shape: (2760, 2)
Final label distribution:
 label
1    1268
0     922
2     570
Name: count, dtype: int64

Sample of cleaned data:
                                                  text  label
473  Осы қазақтар неге сонша тентек? Бәрі бірдей ма...      2
474  Сенің анаңды шешеңмен бірге... п...ц, неге осы...      1
475  Алматыда жаңа кафе ашылған екен, барып көрген ...      0
476  Уйгурлар кетсін, біздің жерімізді басып алды, ...      2
477   Админ ты шо, совсем ебанутый? Пост өшірдің неге?      1


In [ ]:
# Cell 4: Convert to Hugging Face Dataset and Split

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(cleaned_df)

# Split the dataset into training (90%) and testing (10%)
train_test_split_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

# Create a DatasetDict
hf_dataset = DatasetDict({
    'train': train_test_split_dataset['train'],
    'test': train_test_split_dataset['test']
})

print("\nHugging Face Dataset structure:")
print(hf_dataset)


Hugging Face Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2484
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 276
    })
})


In [ ]:
# Cell 5: Setup & Configuration (Upgraded Model)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType

# --- Model & Tokenizer Config (UPGRADED) ---
MODEL_NAME = "xlm-roberta-base" # Using a more powerful, modern model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# --- LoRA Config (STRENGTHENED) ---
# We target more modules to give the model more trainable parameters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value", "key", "dense"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# --- Tokenize Function (from your old Cell 6) ---
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# --- Metrics Function (FIXED: average='macro') ---
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"]

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# --- Training Arguments (STABLE) ---
OUTPUT_DIR = "/content/drive/MyDrive/CSCI594Project"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    save_total_limit=1,
)

# --- Data Collator ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Cell 6: Stratified K-Fold Cross-Validation Training
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
from transformers import Trainer

# --- Custom Trainer (FIXED: class_weights dtype to float32) ---
# This fixes the numerical instability bug.
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Calculate weights inside the trainer, using the device
        train_labels = self.train_dataset["label"]
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_labels),
            y=train_labels
        )
        # THE FIX: Use float32 for stable loss calculation
        weights_dtype = logits.dtype
        class_weights_tensor = torch.tensor(class_weights, dtype=weights_dtype).to(self.model.device)

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# --- K-Fold Setup ---
N_SPLITS = 5  # 5 folds = 80% train, 20% validation each time
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Use the master 'cleaned_df'
X = cleaned_df['text']
y = cleaned_df['label']

all_fold_metrics = [] # Store results from each fold

# --- THE K-FOLD LOOP ---
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print(f"\n--- FOLD {fold+1}/{N_SPLITS} ---")

    # 1. Create train/validation dataframes for this fold
    train_df = cleaned_df.iloc[train_index]
    val_df = cleaned_df.iloc[val_index]

    # 2. Convert to Hugging Face Datasets
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    # 3. Tokenize
    tokenized_train = train_dataset.map(tokenize_function, batched=True).remove_columns(["text"])
    tokenized_val = val_dataset.map(tokenize_function, batched=True).remove_columns(["text"])

    # 4. Re-initialize the Model FOR EACH FOLD
    # This is critical to prevent data leakage.
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(id2label),
        id2label=id2label,
        label2id=label2id,
    )
    peft_model = get_peft_model(model, lora_config)

    # 5. Initialize our Custom Weighted Trainer
    trainer = WeightedTrainer(
        model=peft_model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # 6. Train
    print(f"Starting training for fold {fold+1}...")
    trainer.train()

    # 7. Evaluate and save metrics
    print(f"Evaluating fold {fold+1}...")
    metrics = trainer.evaluate()
    all_fold_metrics.append(metrics)
    print(f"Fold {fold+1} Metrics: {metrics}")

    # 8. Clean up to free VRAM (optional but good practice)
    del model, peft_model, trainer
    torch.cuda.empty_cache()


--- FOLD 1/5 ---


Map:   0%|          | 0/2208 [00:00<?, ? examples/s]

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1301667671.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Starting training for fold 1...


...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1301667671.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Starting training for fold 5...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.080400,0.967085,0.605072,0.644269,0.675431,0.611747
2,0.784800,0.647395,0.692029,0.694576,0.738625,0.706737
3,0.646200,0.583025,0.744565,0.754441,0.774104,0.762803
4,0.612600,0.577177,0.728261,0.732878,0.767397,0.744179
5,0.592500,0.558992,0.742754,0.754345,0.779720,0.761721


Evaluating fold 5...


Fold 5 Metrics: {'eval_loss': 0.558991551399231, 'eval_accuracy': 0.7427536231884058, 'eval_precision': 0.7543448609224731, 'eval_recall': 0.7797203902467059, 'eval_f1': 0.7617207238176285, 'eval_runtime': 5.4488, 'eval_samples_per_second': 101.306, 'eval_steps_per_second': 6.423, 'epoch': 5.0}


In [ ]:
# Cell 7: Report Final Cross-Validation Results

print("\n--- Cross-Validation Complete ---")

# Average the metrics across all folds
avg_f1 = np.mean([m['eval_f1'] for m in all_fold_metrics])
avg_precision = np.mean([m['eval_precision'] for m in all_fold_metrics])
avg_recall = np.mean([m['eval_recall'] for m in all_fold_metrics])
avg_accuracy = np.mean([m['eval_accuracy'] for m in all_fold_metrics])

print(f"\n--- Final Average Metrics ({N_SPLITS}-Fold) ---")
print(f"Average Macro F1: {avg_f1:.4f}")
print(f"Average Macro Precision: {avg_precision:.4f}")
print(f"Average Macro Recall: {avg_recall:.4f}")
print(f"Average Accuracy: {avg_accuracy:.4f}")

print("\nFull metrics per fold:")
for i, m in enumerate(all_fold_metrics):
    print(f"Fold {i+1}: F1={m['eval_f1']:.4f}, Precision={m['eval_precision']:.4f}, Recall={m['eval_recall']:.4f}")


--- Cross-Validation Complete ---

--- Final Average Metrics (5-Fold) ---
Average Macro F1: 0.7447
Average Macro Precision: 0.7370
Average Macro Recall: 0.7597
Average Accuracy: 0.7279

Full metrics per fold:
Fold 1: F1=0.7195, Precision=0.7134, Recall=0.7267
Fold 2: F1=0.7628, Precision=0.7552, Recall=0.7822
Fold 3: F1=0.7360, Precision=0.7265, Recall=0.7491
Fold 4: F1=0.7433, Precision=0.7358, Recall=0.7607
Fold 5: F1=0.7617, Precision=0.7543, Recall=0.7797
